In [62]:
from numpy import genfromtxt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from numpy import *
import matplotlib
import csv
from scipy import sparse
%pylab
%matplotlib inline
from matplotlib import pyplot as plt
import plotly
from plotly.graph_objs import *
from plotly import tools
plotly.tools.set_credentials_file(username='pujaarajan', api_key='7m16vvaw4j')
from scipy import sparse as sp
from sklearn import random_projection
from sklearn import cluster

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


#Import Data

In [53]:
description = genfromtxt('description.csv', delimiter=',')
graph  = genfromtxt('graph.csv', delimiter=',')
social_evolution = genfromtxt('social_and_evolution.csv', delimiter=',')

#CCA FUNCTION

In [54]:
def cca(XA,XB,d):
    XA_c=matrix(XA-average(XA,0))
    XB_c=matrix(XB-average(XB,0))
    sig11=transpose(XA_c)*XA_c
    sig12=transpose(XA_c)*XB_c
    sig22=transpose(XB_c)*XB_c
    sig21=transpose(XB_c)*XA_c
    lambdaA,wA=linalg.eigh(inv(sig11)*sig12*inv(sig22)*sig21)
    ind=argsort(lambdaA)[::-1]
    return XA_c*matrix(wA[:,ind[0:d]])

#CCA 1 ON DESCRIPTION AND SOCIAL EVOLUTION

In [55]:
m=1829
n=8000
X_dat=genfromtxt('description.csv',delimiter=',')
X_c1=sparse.coo_matrix((m,n))
X_c1.data=X_dat[:,2].astype(float64)
X_c1.row=X_dat[:,0].astype(int32)
X_c1.col=X_dat[:,1].astype(int32)
description_X=X_c1.todense()

In [56]:
cca1 = cca(social_evolution,description_X,3)

#CCA 2 ON CCA 1 AND GRAPH

In [57]:
Y=genfromtxt('Y.csv',delimiter=',')

In [58]:
cca2 = cca(cca1,Y,2)

In [60]:
cca2[[12,1419,865,146,1653,1176]]

matrix([[ 0.04340777, -0.08687138],
        [-0.03033224,  0.0930695 ],
        [ 0.10931707,  0.06128257],
        [-0.06675834, -0.0529848 ],
        [ 0.02934979, -0.15807753],
        [ 0.05564288, -0.08499975]])

In [63]:
k_means = cluster.KMeans(n_clusters = 2)

In [64]:
k_means.fit(cca2)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=2, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

array([1, 0, 1, 0, 1, 1], dtype=int32)

#OTHER

In [59]:
cca1_3dim = cca(social_evolution,X,3)

In [88]:
data = go.Scatter3d(
    x=cca1_3dim[:,0],
    y=cca1_3dim[:,1],
    z=cca1_3dim[:,2],
)
fig = go.Figure(data=[data])
py.iplot(fig, filename='simple-3d-scatter')